In [2]:
import os
wd = os.path.normpath(os.getcwd() + '/..')
os.chdir(wd)
os.getcwd()

'/home/hyunsung/Workspace/soccercpd'

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from joblib import Parallel, delayed
from src.myconstants import *

pd.set_option('display.width', 250)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In this code, we generate the aggregated records (formation period records, role period records, and role assignment records) used in formation clustering, role labeling, and other applications from FGP files.

Unfortunately, since we only share the single match data (ID 17985) due to our company's security issue, the following code cannot regenerate the above records when you only use the shared data, i.e. `17985.ugp`. (That is, you can only check how these records are generated by reading this code.)

Instead, we directly share the `form_periods.pkl`, `role_periods.csv`, and `role_records.csv` (resulting files of applying this code to the entire dataset) so that you can reproduce the results of aforementioned tasks in another code files.

### Merging formation period records

In [3]:
formcpd_type = 'gseg_avg'
form_periods_files = os.listdir(f'data/{formcpd_type}/form')
form_periods_files.sort()
form_periods_list = []

for file in form_periods_files:
    path = f'data/{formcpd_type}/form/{file}'
    form_periods_list.append(pd.read_pickle(path))

form_periods = pd.concat(form_periods_list, ignore_index=True)
form_periods.to_pickle(f'data/{formcpd_type}/form_periods.pkl')
form_periods

,activity_id,session,form_period,start_dt,end_dt,duration,coords,edge_mat
0,12864,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-719.0, 1625.0], [-1342.0, 660.0], [948.0, 1...","[[0.0, 0.977, 0.955, 0.2, 0.023, 0.828, 0.257,..."
1,12868,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-833.0, -1647.0], [14.0, 181.0], [-369.0, -1...","[[0.0, 0.338, 0.684, 0.994, 0.165, 0.042, 0.15..."
2,12868,2,2,2020-01-01 15:02:00,2020-01-01 15:50:00,2880.0,"[[-933.0, -1939.0], [-106.0, 618.0], [-231.0, ...","[[0.0, 0.164, 0.86, 0.993, 0.131, 0.036, 0.237..."
3,12870,1,1,2020-01-01 13:30:00,2020-01-01 14:19:00,2940.0,"[[-1408.0, 43.0], [460.0, -1087.0], [-285.0, 5...","[[0.0, 0.22, 0.871, 0.098, 0.367, 0.962, 0.328..."
4,12870,2,2,2020-01-01 14:33:00,2020-01-01 15:23:00,3000.0,"[[-1282.0, -147.0], [290.0, -888.0], [-209.0, ...","[[0.0, 0.416, 0.886, 0.054, 0.192, 0.998, 0.27..."
...,...,...,...,...,...,...,...,...
859,9064,2,3,2020-01-01 16:32:00,2020-01-01 16:51:00,1140.0,"[[-1589.0, 486.0], [-1684.0, -710.0], [-34.0, ...","[[0.0, 0.984, 0.515, 0.04, 0.009, 0.025, 0.813..."
860,9281,1,1,2020-01-01 15:00:00,2020-01-01 15:47:00,2820.0,"[[-1478.0, 113.0], [-1176.0, -1051.0], [1666.0...","[[0.0, 0.982, 0.015, 0.022, 0.016, 0.191, 0.98..."
861,9281,2,2,2020-01-01 16:03:00,2020-01-01 16:51:00,2880.0,"[[-1270.0, -89.0], [-1128.0, -1051.0], [1465.0...","[[0.0, 0.94, 0.052, 0.025, 0.047, 0.184, 0.974..."
862,9285,1,1,2020-01-01 14:01:00,2020-01-01 14:48:00,2820.0,"[[-674.0, 1815.0], [-1457.0, 614.0], [-1564.0,...","[[0.0, 0.992, 0.237, 0.295, 0.148, 0.748, 0.20..."


### Generating role period records

In [4]:
role_period_cols = [
    LABEL_ACTIVITY_ID, LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_ROLE_PERIOD,
    LABEL_SESSION, LABEL_START_TIME, LABEL_END_TIME
]
activity_records = pd.read_csv('data/activity_records.csv', header=0, encoding='utf-8-sig')
form_periods = pd.read_pickle(f'data/{formcpd_type}/form_periods.pkl')

In [5]:
def generate_role_period_records(fgp_file, formcpd_type='gseg_avg'):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{formcpd_type}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')

    grouped = fgp_df.groupby(LABEL_ROLE_PERIOD)
    match_role_periods = grouped[[LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION]].first()
    match_role_periods[LABEL_ACTIVITY_ID] = activity_id
    match_role_periods[LABEL_START_TIME] = grouped[LABEL_GAMETIME].min().apply(lambda x: int(x[:2]))
    match_role_periods[LABEL_END_TIME] = grouped[LABEL_GAMETIME].max().apply(lambda x: int(x[:2]))
    match_role_periods.reset_index(inplace=True)

    return match_role_periods[role_period_cols]

In [6]:
formcpd_type = 'gseg_avg'
fgp_files = os.listdir(f'data/{formcpd_type}/fgp')
role_period_list = Parallel(n_jobs=50)(
    delayed(generate_role_period_records)(f, formcpd_type) for f in tqdm(fgp_files)
)
role_periods = pd.concat(role_period_list, axis=0, ignore_index=True).astype(int)
role_periods.sort_values([LABEL_ACTIVITY_ID, LABEL_ROLE_PERIOD], inplace=True)
role_periods

100%|██████████| 457/457 [00:03<00:00, 144.73it/s]


,activity_id,player_period,form_period,role_period,session,start_time,end_time
763,1879,1,1,1,1,0,35
764,1879,2,1,2,1,35,47
765,1879,3,2,3,2,0,31
766,1879,4,2,4,2,31,42
767,1879,4,2,5,2,42,49
...,...,...,...,...,...,...,...
1780,22875,2,1,2,1,29,40
1781,22875,2,1,3,1,40,47
1782,22875,3,2,4,2,0,18
1783,22875,4,2,5,2,18,49


In [7]:
role_periods.to_csv(f'data/{formcpd_type}/role_periods.csv', index=False)

### Generating and merging role assignment records

In [4]:
role_record_cols = [
    LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION,
    LABEL_DATETIME, LABEL_SQUAD_NUM, LABEL_PLAYER_NAME, LABEL_BASE_ROLE
]

In [10]:
def generate_role_records(fgp_file, formcpd_type='gseg_avg', freq=1):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{formcpd_type}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')
    fgp_df[LABEL_DATETIME] = fgp_df[LABEL_DATETIME].apply(lambda dt: datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'))

    grouped = fgp_df.groupby([LABEL_PLAYER_ID, LABEL_ROLE_PERIOD], as_index=False)
    role_records = grouped[role_record_cols].first()
    role_records[LABEL_ACTIVITY_ID] = activity_id
    role_records[LABEL_START_DT] = grouped[LABEL_DATETIME].first()[LABEL_DATETIME] - timedelta(seconds=freq)
    role_records[LABEL_END_DT] = grouped[LABEL_DATETIME].last()[LABEL_DATETIME]
    role_records[LABEL_DURATION] = grouped[LABEL_GAMETIME].count()[LABEL_GAMETIME] * freq

    role_records = pd.merge(role_records, form_periods[[LABEL_ACTIVITY_ID, LABEL_FORM_PERIOD, LABEL_COORDS]])
    role_records[LABEL_X] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 0], axis=1)
    role_records[LABEL_Y] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 1], axis=1)

    role_records = role_records[HEADER_ROLE_RECORDS].sort_values(
        [LABEL_SQUAD_NUM, LABEL_ROLE_PERIOD], ignore_index=True)
    target_path = f'data/{formcpd_type}/role/{activity_id}.csv'
    role_records.to_csv(target_path, index=False, encoding='utf-8-sig')

    return target_path

In [15]:
formcpd_type = 'gseg_avg'
fgp_files = os.listdir(f'data/{formcpd_type}/fgp')
target_paths = Parallel(n_jobs=50)(delayed(generate_role_records)(f, formcpd_type) for f in tqdm(fgp_files))
target_paths.sort()
pd.read_csv(target_paths[0], header=0)

100%|██████████| 457/457 [00:08<00:00, 51.63it/s] 


,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,4779,3,P03,1,-719.0,1625.0
1,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1925,4,P04,2,-1342.0,660.0
2,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1248,7,P07,3,948.0,1576.0
3,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1759,8,P08,4,-180.0,-451.0
4,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1252,9,P09,5,1574.0,-572.0
5,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5713,10,P10,6,48.0,543.0
6,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5439,15,P15,7,-1313.0,-551.0
7,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1235,23,P23,8,-982.0,-1634.0
8,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,6088,72,P72,9,584.0,-1632.0
9,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1773,98,P98,10,1383.0,436.0


In [19]:
role_records_files = os.listdir(f'data/{formcpd_type}/role')
role_records_files.sort()
role_records_list = []

for file in role_records_files:
    path = f'data/{formcpd_type}/role/{file}'
    role_records_list.append(pd.read_csv(path, header=0))

role_records = pd.concat(role_records_list, ignore_index=True)
for col in HEADER_ROLE_RECORDS[:5]:
    role_records[col] = role_records[col].astype(int)

role_records.to_csv(f'data/{formcpd_type}/role_records.csv', index=False)
role_records

,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,4779,3,P03,1,-719.0,1625.0
1,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1925,4,P04,2,-1342.0,660.0
2,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1248,7,P07,3,948.0,1576.0
3,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1759,8,P08,4,-180.0,-451.0
4,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1252,9,P09,5,1574.0,-572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21123,9285,2,2,2,2,2020-01-01 15:03:00,2020-01-01 15:08:50,350,1755,77,P77,8,1003.0,1728.0
21124,9285,2,2,3,2,2020-01-01 15:08:50,2020-01-01 15:31:00,1330,1755,77,P77,10,1255.0,-1739.0
21125,9285,3,2,4,2,2020-01-01 15:31:00,2020-01-01 15:35:00,240,1755,77,P77,10,1255.0,-1739.0
21126,9285,4,2,5,2,2020-01-01 15:35:00,2020-01-01 15:49:00,840,1755,77,P77,10,1255.0,-1739.0
